# 导入需要的包

In [1]:
import lightgbm as lgb

# 数据接口

LightGBM Python 模块能够使用以下几种方式来加载数据:
+ libsvm/tsv/csv txt format file（libsvm/tsv/csv 文本文件格式）
+ Numpy 2D array, pandas object（Numpy 2维数组, pandas 对象）
+ LightGBM binary file（LightGBM 二进制文件）

```python
# 加载文件
train_data = lgb.Dataset('train.svm.bin')
# 加载numpy 数组到 Dataset 中
data = np.random.rand(500, 10)  # 500 个样本, 每一个包含 10 个特征
label = np.random.randint(2, size=500)  # 二元目标变量,  0 和 1
train_data = lgb.Dataset(data, label=label)
#加载 scpiy.sparse.csr_matrix 数组到 Dataset 中
csr = scipy.sparse.csr_matrix((dat, (row, col)))
train_data = lgb.Dataset(csr)
#保存 Dataset 到 LightGBM 二进制文件将会使得加载更快速
train_data = lgb.Dataset('train.svm.txt')
train_data.save_binary('train.bin')

# 创建验证数据
1）test_data = train_data.create_valid('test.svm')
2）test_data = lgb.Dataset('test.svm', reference=train_data)


```

# 设置参数

```python
# 将参数写成字典形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression', # 目标函数     ####regression默认regression_l2
    'metric': {'l2', 'auc'},  # 评估函数
    'max_depth': 6     ###   树的深度           ###按层
    'num_leaves': 50  ###   由于leaves_wise生长，小于2^max_depth   #####按leaf_wise
    'learning_rate': 0.05,  # 学习速率
    'subsample'/'bagging_fraction':0.8           ###  数据采样
    'colsample_bytree'/'feature_fraction': 0.8  ###  特征采样
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
```

# 参数详解
https://lightgbm.readthedocs.io/en/latest/Python-API.html

# 调参方法

调参：https://www.imooc.com/article/43784?block_id=tuijian_wz

LightGBM的调参过程和RF、GBDT等类似，其基本流程如下：
+ 首先选择较高的学习率，大概0.1附近，这样是为了加快收敛的速度。这对于调参是很有必要的。
+ 对决策树基本参数调参：
   - 1）max_depth和num_leaves
   - 2）min_data_in_leaf和min_sum_hessian_in_leaf
   - 3）feature_fraction和bagging_fraction
+ 正则化参数调参
+ 最后降低学习率，这里是为了最后提高准确率

# 实例

## 原生接口

In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
 
iris = load_iris()
data = iris.data
target = iris.target
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
 
# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
 
# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression',  # 目标函数
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 31,  # 叶子节点数
    'learning_rate': 0.05,  # 学习速率
    'feature_fraction': 0.9,  # 建树的特征选择比例
    'bagging_fraction': 0.8,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
 
# 训练 cv and train
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=5)
 
# 保存模型到文件
gbm.save_model('model.txt')
 
# 预测数据集
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
 
# 评估模型
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000044 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score 1.025000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 1	valid_0's l2: 0.702763
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 1	valid_0's l2: 0.640034
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 1	valid_0's l2: 0.583441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 1	valid_0's l2: 0.532384
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 1	valid_0's l2: 0.486322
[Ligh

## sklearn接口实例

In [3]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
 
# 加载数据
iris = load_iris()
data = iris.data
target = iris.target
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
 
# 创建模型，训练模型
gbm = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=20)
gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l1', early_stopping_rounds=5)
 
# 测试机预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
 
# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
 
# feature importances
print('Feature importances:', list(gbm.feature_importances_))
 
# 网格搜索，参数优化
estimator = lgb.LGBMRegressor(num_leaves=31)
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}
gbm = GridSearchCV(estimator, param_grid)
gbm.fit(X_train, y_train)
print('Best parameters found by grid search are:', gbm.best_params_)

[1]	valid_0's l1: 0.591321	valid_0's l2: 0.561313
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 0.568299	valid_0's l2: 0.517453
[3]	valid_0's l1: 0.546427	valid_0's l2: 0.477944
[4]	valid_0's l1: 0.525649	valid_0's l2: 0.442359
[5]	valid_0's l1: 0.505911	valid_0's l2: 0.410312
[6]	valid_0's l1: 0.487158	valid_0's l2: 0.381454
[7]	valid_0's l1: 0.469344	valid_0's l2: 0.355471
[8]	valid_0's l1: 0.45242	valid_0's l2: 0.33208
[9]	valid_0's l1: 0.436343	valid_0's l2: 0.311025
[10]	valid_0's l1: 0.422146	valid_0's l2: 0.293244
[11]	valid_0's l1: 0.408659	valid_0's l2: 0.277265
[12]	valid_0's l1: 0.39485	valid_0's l2: 0.261816
[13]	valid_0's l1: 0.37997	valid_0's l2: 0.243512
[14]	valid_0's l1: 0.368479	valid_0's l2: 0.231915
[15]	valid_0's l1: 0.355278	valid_0's l2: 0.216033
[16]	valid_0's l1: 0.345354	valid_0's l2: 0.206682
[17]	valid_0's l1: 0.334284	valid_0's l2: 0.193469
[18]	valid_0's l1: 0.325227	valid_0's l2: 0.185919
[19]	valid_0's l1: 0.315843	valid_0

# 原生API与sklearnAPI接口区别总结
1.多分类时lgb.train除了’objective’:‘multiclass’,还要指定"num_class":5，而sklearn接口只需要指定’objective’:‘multiclass’。

2.lgb.train中正则化参数为"lambda_l1", “lambda_l1”，sklearn中则为’reg_alpha’, ‘reg_lambda’。

3.迭代次数在sklearn中是’n_estimators’:300，在初始化模型时指定。而在lgb.train中则可在参数params中指定，也可在函数形参中指出。